In [20]:
import pandas as pd
import datetime
import pdb

In [2]:
# This is the data that ChatGPT gave me.  Clean up NULLs and the format of the date
data_from_chat_gpt = pd.read_csv('./calendar_from_chat_gpt.csv')
data_from_chat_gpt = data_from_chat_gpt.where(pd.notnull(data_from_chat_gpt), None)
data_from_chat_gpt['date'] = pd.to_datetime(data_from_chat_gpt['date'])
data_from_chat_gpt['date'] = data_from_chat_gpt['date'].dt.date
data_from_chat_gpt.head(15)

,date,color
0,2025-08-18,red
1,2025-08-19,orange
2,2025-08-20,yellow
3,2025-08-21,green
4,2025-08-22,blue
5,2025-08-25,purple
6,2025-08-26,red
7,2025-08-27,orange
8,2025-08-28,yellow
9,2025-08-29,green


In [3]:
data_from_chat_gpt.dtypes

date     object
color    object
dtype: object

In [ ]:
# First thing I want to do is assign a letter to the days.  I believe there is a direct correlation between color and letter as follows
COLOR_LETTER_CONVERT = dict(red='a', orange='b', yellow='c', green='d', blue='e', purple='f')

# And we are always expecting the colors to exactly rotate in this pattern
COLOR_ORDER = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']

In [21]:
essentials_calendar = []
previous_date = None
previous_letter = None
MAX_ITERS = 20000

for i, row in enumerate(data_from_chat_gpt.to_dict('records')):
    if i >= MAX_ITERS:
        break

    current_date = row['date']

    row['letter'] = COLOR_LETTER_CONVERT[row['color']] if row['color'] and COLOR_LETTER_CONVERT[row['color']] else None
    if previous_letter and row['letter'] is not None:
        if previous_letter == 'f':
            if row['letter'] != 'a':
                raise Exception(f"Color order is wrong at {row['date']}")
            else:
                if row['letter'] != chr(ord(previous_letter) + 1):
                    raise Exception(f"Color order is wrong at {row['date']}")
        previous_letter = row['letter']

    # Check if current_date is a weekday (Monday=0, Sunday=6)
    if current_date.weekday() < 5:  # 0-4 are weekdays
        if previous_date is None:
            essentials_calendar.append(row)
            previous_date = current_date
        else:
            # Calculate expected next weekday

            expected_next = previous_date + datetime.timedelta(days=1)
            while expected_next.weekday() > 4:
                expected_next += datetime.timedelta(days=1)
            if current_date == expected_next:
                essentials_calendar.append(row)
                previous_date = current_date
    else:
        raise Exception("Found a weekend in the gpt data")

# Convert to DataFrame
essentials_calendar_df = pd.DataFrame(essentials_calendar)
essentials_calendar_df.head()

,date,color,letter
0,2025-08-18,red,a
1,2025-08-19,orange,b
2,2025-08-20,yellow,c
3,2025-08-21,green,d
4,2025-08-22,blue,e


In [22]:
# Use this data in the flask app
essentials_calendar_df.to_csv('../flask_app/data/essentials_calendar.csv', index=False)

In [23]:
essentials_calendar_df = essentials_calendar_df.set_index('date')

In [28]:
search_date = pd.to_datetime('2024-09-04').date()
search_date

datetime.date(2024, 9, 4)

In [29]:
essentials_calendar_df.loc[search_date]

KeyError: datetime.date(2024, 9, 4)